<a href="https://colab.research.google.com/github/CheayeonLee/ESAA_OB/blob/main/3%EC%A3%BC%EC%B0%A8%EA%B3%BC%EC%A0%9C(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블 학습과 랜덤 포레스트

## 7.1 투표 기반 분류기
- 직접 투표(hard voting): 다수결 투표로 정해지는 분류기
- 큰 수의 법칙: ex.동전을 자꾸 던질수록 앞면이 나오는 비율은 점점 더 앞면이 나올 확률에 가까워짐

앙상블 방법
- 예측기가 가능한 한 서로 독립적일 때 최고의 성능을 발휘
- 각기 다른 알고리즘으로 학습 시키는 것이 좋음
- 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도를 향상시킴

In [2]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.15)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [4]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8333333333333334
RandomForestClassifier 0.9666666666666667
SVC 0.9333333333333333
VotingClassifier 0.9333333333333333


간접 투표(soft voting): 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측
- SVC는 기본값에서는 클래스 확률을 제공하지 않으므로 probability 매개변수를 True로 지정해야 함

## 7.2 배깅과 페이스팅

- 배깅: 훈련 세트에서 중복을 허용하여 샘플링하는 방식
- 페이스팅:  중복을 허용하지 않고 샘플링하는 방식

- 통계적 최빈값: 직접 투표 분류기처럼 가장 많은 예측 결과로 분류

### 7.2.1 사이킷런의 배깅과 페이스팅

In [16]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1) # n_jobs 매개변수는 사이킷런이 훈련과 예측에 사용할 CPU 코어 수를 지정
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

부트스트래핑
- 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 조금 더 높음
- 다양성 추가는 예측기들의 상관관계를 줄이므로 앙상블의 분산 감소

### 7.2.2 oob 평가

- 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있음
- oob(out-of-bag)샘플: 선택되지 않은 훈련 샘플의 나머지 37%

In [17]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True  # 평가
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9285714285714286

- BaggingClassifier는 test 세트에서 약 92%의 정확도를 얻을 것으로 보임

In [18]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9333333333333333

In [19]:
# oob_decision_function_ : oob 샘플에 대한 결정 함수의 값
# ex. 2번째 훈련 샘플이 양성 클래스에 속할 확률은 19%
bag_clf.oob_decision_function_

array([[0.98802395, 0.01197605],
       [0.        , 1.        ],
       [0.12105263, 0.87894737],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.98795181, 0.01204819],
       [0.94535519, 0.05464481],
       [0.02673797, 0.97326203],
       [0.07602339, 0.92397661],
       [0.88947368, 0.11052632],
       [0.76086957, 0.23913043],
       [0.        , 1.        ],
       [0.80745342, 0.19254658],
       [0.00502513, 0.99497487],
       [0.58959538, 0.41040462],
       [0.98974359, 0.01025641],
       [1.        , 0.        ],
       [0.03448276, 0.96551724],
       [0.73493976, 0.26506024],
       [1.        , 0.        ],
       [0.02285714, 0.97714286],
       [0.58045977, 0.41954023],
       [0.50537634, 0.49462366],
       [0.04519774, 0.95480226],
       [0.98857143, 0.01142857],
       [0.02538071, 0.97461929],
       [0.02659574, 0.97340426],
       [0.74093264, 0.25906736],
       [0.        , 1.        ],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링

## 7.4 랜덤 포레스트

In [20]:
# max_samples가 전형적
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [23]:
# 랜덤포레스트와 거의 유사하게 만든 것
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16),
                           n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

### 7.4.1 엑스트라 트리

- 익스트림 랜덤 트리: 극단적으로 무작위한 트리의 랜덤 포레스트
- 편향이 늘어나지만 대신 분산을 낮추게 됨

### 7.4.2 특성 중요도

- 랜덤포레스트의 장점: 특성의 상대적 중요도를 측정하기 쉬움
- 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화

In [24]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09274189095614499
sepal width (cm) 0.024209261585693136
petal length (cm) 0.40634017690942087
petal width (cm) 0.4767086705487409


## 7.5 부스팅

- 부스팅: 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 것
- 인기 1. AdaBOost 2. Gradient Boosting

### 7.5.1 에이다부스트

- 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것
- 학습하기 어려운 샘플에 점점 더 맞춰지게 됨

In [25]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레이디언트 부스팅

- 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습

In [26]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [27]:
# 첫 번째 예측기에서 생긴 잔여 오차에 훈련시킴
y2 = y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [28]:
# 두 번째 예측기에서 생긴 잔여 오차에 훈련시킴
y3 = y2-tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [30]:
y_pred = sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [31]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [34]:
# staged_predict(): 최적의 트리 수를 찾기 위한 조기 종료 기법
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=102)

In [37]:
# warm_start=True: fit()메서드가 호출될 때 기존 트리를 유지하고 추가할 수 있도록 해줌
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up += 1
    if error_going_up == 5:
      break # 조기 종료

XGBoost
- 빠른 속도, 확장성, 이식성이 목표

In [38]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [39]:
xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.38267
[1]	validation_0-rmse:0.32489
[2]	validation_0-rmse:0.26550
[3]	validation_0-rmse:0.23131
[4]	validation_0-rmse:0.21156
[5]	validation_0-rmse:0.19082
[6]	validation_0-rmse:0.18137
[7]	validation_0-rmse:0.17032
[8]	validation_0-rmse:0.17066
[9]	validation_0-rmse:0.16920
[10]	validation_0-rmse:0.16791
[11]	validation_0-rmse:0.16497
[12]	validation_0-rmse:0.16344
[13]	validation_0-rmse:0.16208
[14]	validation_0-rmse:0.16099
[15]	validation_0-rmse:0.16016
[16]	validation_0-rmse:0.15939
[17]	validation_0-rmse:0.15876
[18]	validation_0-rmse:0.15794
[19]	validation_0-rmse:0.15749
[20]	validation_0-rmse:0.15755


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[21]	validation_0-rmse:0.15765


## 7.6 스태킹

- '앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까?'에서 시작
- 사이킷런에서 직접 지원하진 않음